<a href="https://colab.research.google.com/github/Ayanlola2002/Stutern/blob/master/Stutern_Group_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [159]:
import pandas as pd
all_accounts=pd.read_csv('/content/drive/My Drive/Stutern/Group1_task/raw/all_accounts.csv')
all_balances=pd.read_csv('/content/drive/My Drive/Stutern/Group1_task/raw/all_balances.csv')
all_demographics=pd.read_csv('/content/drive/My Drive/Stutern/Group1_task/raw/all_demographics.csv',encoding='latin',sep='|')
all_investments=pd.read_csv('/content/drive/My Drive/Stutern/Group1_task/raw/all_investments.csv')
all_loans=pd.read_csv('/content/drive/My Drive/Stutern/Group1_task/raw/all_loans.csv')
all_loans_application=pd.read_csv('/content/drive/My Drive/Stutern/Group1_task/raw/all_loans_application.csv')
all_transactions_digital=pd.read_csv('/content/drive/My Drive/Stutern/Group1_task/raw/all_transactions_digital.csv')
bureau_score=pd.read_csv('/content/drive/My Drive/Stutern/Group1_task/raw/bureau_score.csv')
all_dictionary=pd.read_csv('/content/drive/My Drive/Stutern/Group1_task/raw/dictionary.csv',skiprows=[0])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,19,26,28,30,35,36,37,38,39,61,71,73,74,85,100,103,114,115,125,126) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [160]:
#we use the all_loans_applications customer id 
#this indicates all those customers that apply for loans 
#it may also be possible that a customer ask for multiple loans
print(all_loans.shape)
print(all_loans_application.shape)

(278505, 10)
(23172, 8)


In [0]:
all_loans_application_data=all_loans_application.copy()

In [0]:
#setting customerId as index 
all_loans_application.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [0]:
#extracting customer ID from  all_loan_applications
important_indexes=all_loans_application.index.unique()

In [0]:
#resetting of all_accounts index
all_accounts.set_index('CUSTOMER_UNIQUE_ID',inplace=True)


In [165]:
all_accounts=all_accounts.loc[important_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [166]:
all_accounts['RECORD_STAT'].value_counts()

O    27281
C     1776
Name: RECORD_STAT, dtype: int64

In [167]:
all_accounts.shape

(29068, 4)

In [0]:
all_accounts.drop(columns='MASKED_ACCOUNT',inplace=True)

In [169]:
#checking different type of currency
all_accounts['CCY'].value_counts()

NGN    26765
USD     2082
GBP      146
EUR       64
Name: CCY, dtype: int64

In [0]:
#function to convert different currency to naira
def con(data):
    data.loc[data.CCY=='USD','ACY_AVL_BAL']=data.loc[data.CCY=='USD','ACY_AVL_BAL'].apply(lambda x: x*365)
    data.loc[data.CCY=='EUR','ACY_AVL_BAL']=data.loc[data.CCY=='EUR','ACY_AVL_BAL'].apply(lambda x: x*398)
    data.loc[data.CCY=='GBP','ACY_AVL_BAL']=data.loc[data.CCY=='GPB','ACY_AVL_BAL'].apply(lambda x: x*467)
    data.loc[data.CCY=='JPY','ACY_AVL_BAL']=data.loc[data.CCY=='JPY','ACY_AVL_BAL'].apply(lambda x: x*3.31)
    return data

In [0]:
#changing all account to naira notes
all_accounts=con(all_accounts)

In [0]:
#convert index to int
all_accounts.index=all_accounts.index.astype('int')

In [0]:
#sort index
all_accounts.sort_index(inplace=True)

In [174]:
#meaning of ACY_AVL_BAL
all_dictionary[all_dictionary['FEATURES']=='ACY_AVL_BAL']

,FEATURES,DESCRIPTION,DATA TYPE,Unnamed: 3,Unnamed: 4
14,ACY_AVL_BAL,Specifies the available balance in the account...,NUMBER,NaN,NaN


In [175]:
#some of the ACY_AVL_BAL IS negative we will assumed it is zero 
all_accounts['ACY_AVL_BAL'].value_counts()

 0.00          12405
-52.50           243
-105.00           69
 950.00           33
 1.00             26
               ...  
 77773.41          1
 4058776.43        1
 125.53            1
 14164.08          1
-145.34            1
Name: ACY_AVL_BAL, Length: 14389, dtype: int64

In [0]:
#for all ACY_AVL_BAL that is negative RECORD STAT IS C we convert to zero so as not to affect our model
all_accounts.loc[(all_accounts.ACY_AVL_BAL!=0) & (all_accounts.RECORD_STAT=="C"),'ACY_AVL_BAL']=0

In [177]:
all_accounts.index.value_counts()

69090004     34
578029002    27
172877006    25
657601000    23
679735005    22
             ..
952341006     1
73406011      1
234473009     1
806118002     1
909990007     1
Name: CUSTOMER_UNIQUE_ID, Length: 17986, dtype: int64

In [178]:
#most of the customer id appears more than ones so we can group by customer id like the example below
all_accounts.loc[69090004]

,RECORD_STAT,ACY_AVL_BAL,CCY
CUSTOMER_UNIQUE_ID,,,
69090004,O,0.00,USD
69090004,C,0.00,NGN
69090004,O,37128.66,NGN
69090004,O,0.00,NGN
69090004,O,28.09,NGN
69090004,O,0.00,NGN
69090004,O,0.00,NGN
69090004,O,1262799.23,NGN
69090004,O,89483.40,USD


In [0]:
#currency was removed because all ACY_AVL_BAL was coverted to same currency naira
all_accounts_data=all_accounts.groupby([all_accounts.index])[['ACY_AVL_BAL']].sum()

In [180]:
all_accounts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29068 entries, 56000 to 999944001
Data columns (total 3 columns):
RECORD_STAT    29057 non-null object
ACY_AVL_BAL    28938 non-null float64
CCY            29057 non-null object
dtypes: float64(1), object(2)
memory usage: 1.5+ MB


In [181]:
#there will be no need to merge CCY column to all_accounts dataframe since we converted all currency to naira.
#all_accounts_data treated
all_accounts_data.head()

,ACY_AVL_BAL
CUSTOMER_UNIQUE_ID,
56000,136043.48
62005,1541719.95
104008,12364.06
175007,20029.55
200006,61489.56


In [0]:
all_balances.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [183]:
#focussing on only those who made loan applications according to all_loan_applications
all_balances=all_balances.loc[important_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [184]:
#checking for null values
all_balances.isnull().sum()

MASKED_ACCOUNT    490
VAL_DT            490
BAL               490
dtype: int64

In [0]:
#trying to select the last date balance of customers who applied for loans
#merger=all_balances.groupby([all_balances.index,all_balances.MASKED_ACCOUNT])[['VAL_DT']].max()

In [0]:
all_balances_data=all_balances.sort_values('VAL_DT').groupby(all_balances.index).tail(1)

In [187]:
all_balances_data.head()

,MASKED_ACCOUNT,VAL_DT,BAL
CUSTOMER_UNIQUE_ID,,,
41981006,00******68,10/1/2019,40821.02
737790008,06******08,10/1/2019,323.75
521640001,00******56,10/1/2019,11974.56
391254009,00******30,10/1/2019,6813.05
657601000,00******15,10/1/2019,14998.88


In [188]:
all_balances_data.isnull().any(axis=1).sum()

4

In [189]:
#drop rows with na values
all_balances_data.dropna(how='any',axis=0)

,MASKED_ACCOUNT,VAL_DT,BAL
CUSTOMER_UNIQUE_ID,,,
41981006,00******68,10/1/2019,40821.02
737790008,06******08,10/1/2019,323.75
521640001,00******56,10/1/2019,11974.56
391254009,00******30,10/1/2019,6813.05
657601000,00******15,10/1/2019,14998.88
...,...,...,...
801451003,07******68,9/9/2019,85273.88
588148009,00******46,9/9/2019,1097.83
13231006,00******65,9/9/2019,40002.64


In [190]:
#rows with null values
all_balances_data.loc[all_balances_data['VAL_DT'].isnull()]

,MASKED_ACCOUNT,VAL_DT,BAL
CUSTOMER_UNIQUE_ID,,,
909766003,NaN,NaN,NaN
297233009,NaN,NaN,NaN
311246013,NaN,NaN,NaN
162998000,NaN,NaN,NaN


In [0]:
#drop row index with null values
all_balances_data=all_balances_data.drop([909766003,297233009,311246013,162998000])

In [192]:
#all_balances_data treated
all_balances_data.isna().sum()

MASKED_ACCOUNT    0
VAL_DT            0
BAL               0
dtype: int64

In [193]:
#removing masked account Masked_account and val_dt
all_balances_data.drop(['MASKED_ACCOUNT','VAL_DT'],axis=1)

,BAL
CUSTOMER_UNIQUE_ID,
41981006,40821.02
737790008,323.75
521640001,11974.56
391254009,6813.05
657601000,14998.88
...,...
801451003,85273.88
588148009,1097.83
13231006,40002.64


In [0]:
#convert index to int
all_balances_data.index=all_balances_data.index.astype('int')

In [0]:
all_balances_data.sort_index(inplace=True)

In [0]:
#using regex101 online tool on all_demographics
all_demographics.ADDRESS_LINE3=all_demographics.ADDRESS_LINE3.str.replace(',\s*','')
all_demographics.ADDRESS_LINE3=all_demographics.ADDRESS_LINE3.str.replace('\d*','')

In [197]:
all_demographics.head()

,CUSTOMER_UNIQUE_ID,CUSTOMER_TYPE,ADDRESS_LINE3,ADDRESS_LINE4,COUNTRY,NATIONALITY,LANGUAGE,EXPOSURE_COUNTRY,LOCAL_BRANCH,FROZEN,DECEASED,WHEREABOUTS_UNKNOWN,CUSTOMER_CATEGORY,HO_AC_NO,FX_CUST_CLEAN_RISK_LIMIT,OVERALL_LIMIT,FX_CLEAN_RISK_LIMIT,CREDIT_RATING,LIMIT_CCY,CAS_CUST,LIAB_NODE,SEC_CUST_CLEAN_RISK_LIMIT,SEC_CLEAN_RISK_LIMIT,SEC_CUST_PSTL_RISK_LIMIT,SEC_PSTL_RISK_LIMIT,LIAB_BR,PAST_DUE_FLAG,DEFAULT_MEDIA,SSN,SWIFT_CODE,LOC_CODE,UTILITY_PROVIDER,UTILITY_PROVIDER_ID,RISK_PROFILE,DEBTOR_CATEGORY,UDF_1,UDF_2,UDF_3,UDF_4,UDF_5,...,PINCODE,MFI_CUSTOMER,CUSTOMER_PREFIX,DATE_OF_BIRTH,MINOR,SEX,RESIDENT_STATUS,AGE_PROOF_SUBMITTED,CUST_COMM_MODE,PLACE_OF_BIRTH,BIRTH_COUNTRY,US_RES_STATUS,VST_US_PREV,P_PINCODE,PREF_CONTACT_DT,PREF_CONTACT_TIME,D_PINCODE,EDUCATIONAL_STATUS,MARITAL_STATUS,SPOUSE_NAME,SPOUSE_EMP_STATUS,DEPENDENT_CHILDREN,DEPENDENT_OTHERS,ACCOMODATION,EMPLOYMENT_STATUS,EMPLOYMENT_TENURE,RETIREMENT_AGE,PREV_DESIGNATION,PREV_EMPLOYER,DESIGNATION,SALARY,OTHER_INCOME,RENT,INSURANCE,LOAN_PAYMENT,OTHER_EXPENSES,HOUSE_VALUE,CREDIT_CARDS,E_COUNTRY,CCY_PERS_INCEXP
0,18030000,I,IKOYI,LAGOS,NG,NG,ENG,NG,3,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,3,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23401,N,MR,1966-08-15 00:00:00,N,M,R,N,E,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33020001,I,IKORODU,LAGOS,NG,NG,ENG,NG,6,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,6,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,MR,1950-01-01 00:00:00,N,M,R,N,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69090004,I,Lagos State,", VICTORIA ISLAND",NG,NG,ENG,NG,14,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,14,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,N,Ms.,1970-04-10 00:00:00,N,F,R,N,E,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,81010006,I,SHAGAMU ROADIKORODU,LAGOS,NG,NG,ENG,NG,6,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,6,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,NaN,MR,1967-04-26 00:00:00,N,M,R,N,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71120007,I,SURULERE,LAGOS,NG,NG,ENG,NG,4,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,4,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,N,MR,1965-09-24 00:00:00,N,M,R,N,M,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#dealing with line 4
all_demographics.ADDRESS_LINE4=all_demographics.ADDRESS_LINE4.str.replace(',\s*','')
all_demographics.ADDRESS_LINE4=all_demographics.ADDRESS_LINE4.str.replace('\d*','')

In [0]:
#to csv
all_demographics.to_csv('all_demographics2.csv')

In [200]:
all_demographics.ADDRESS_LINE4.value_counts().to_dict()

{'': 2695,
 ' ': 26,
 ' ABUJA': 1,
 ' IJEGUN LAGOS': 1,
 ' LAGOS': 2,
 ' OFF POWER ENCOUNTER ROADPHC.': 1,
 ' OGUN STATE': 1,
 ' VICTORIA ISLAND': 1,
 ' ZAMFARA STATE': 1,
 '(COLLECT BY HAND)': 1,
 '(H)': 3,
 '(HOLD ALL CORRESPONDENCES)': 1,
 '(HOLD)': 2,
 '(SEND)': 2,
 '(TO BE COLLECTED)': 3,
 '.': 2,
 'ABA': 1,
 'ABA                 ': 2,
 'ABAI': 1,
 'ABAKALIKI ': 1,
 'ABANDAN': 1,
 'ABARANJE ROADIKOTUN.IKOTUN': 1,
 'ABEOKUTA': 1,
 'ABEOKUTA CLSE AREA  A GARKI': 1,
 'ABEOKUTA.': 1,
 'ABIA': 86,
 'ABUJA': 52,
 'ABUJA ': 1,
 'ABUJA FCT': 1,
 'ABUJA.': 4,
 'AC/O-ONYEMA/UGO': 1,
 'ADAMAWA': 58,
 'ADAMAWA.': 1,
 'ADO EKITI': 2,
 'AGBARA': 1,
 'AGO-OKOTALAGOS.': 1,
 'AJAH LAGOS': 1,
 'AKOKO EDO': 1,
 'AKOWONJO LAGOS': 1,
 'AKURE': 1,
 'AKUTE': 1,
 'AKUTEOGUN STATE.': 1,
 'AKWA  IBOM': 1,
 'AKWA IBOM': 119,
 'AKWA IBOM STATE': 2,
 'AKWA IBOM STATE.': 1,
 'AKWA UBIM': 1,
 'AKWA-IBOM': 4,
 'AKWANGA ': 1,
 'ALAKIAIBADAN': 1,
 'ALLEN': 1,
 'ANAMBRA': 90,
 'ANAMBRA STATE.': 1,
 'ANAMBRA.': 1,
 

In [201]:
all_demographics.head()

,CUSTOMER_UNIQUE_ID,CUSTOMER_TYPE,ADDRESS_LINE3,ADDRESS_LINE4,COUNTRY,NATIONALITY,LANGUAGE,EXPOSURE_COUNTRY,LOCAL_BRANCH,FROZEN,DECEASED,WHEREABOUTS_UNKNOWN,CUSTOMER_CATEGORY,HO_AC_NO,FX_CUST_CLEAN_RISK_LIMIT,OVERALL_LIMIT,FX_CLEAN_RISK_LIMIT,CREDIT_RATING,LIMIT_CCY,CAS_CUST,LIAB_NODE,SEC_CUST_CLEAN_RISK_LIMIT,SEC_CLEAN_RISK_LIMIT,SEC_CUST_PSTL_RISK_LIMIT,SEC_PSTL_RISK_LIMIT,LIAB_BR,PAST_DUE_FLAG,DEFAULT_MEDIA,SSN,SWIFT_CODE,LOC_CODE,UTILITY_PROVIDER,UTILITY_PROVIDER_ID,RISK_PROFILE,DEBTOR_CATEGORY,UDF_1,UDF_2,UDF_3,UDF_4,UDF_5,...,PINCODE,MFI_CUSTOMER,CUSTOMER_PREFIX,DATE_OF_BIRTH,MINOR,SEX,RESIDENT_STATUS,AGE_PROOF_SUBMITTED,CUST_COMM_MODE,PLACE_OF_BIRTH,BIRTH_COUNTRY,US_RES_STATUS,VST_US_PREV,P_PINCODE,PREF_CONTACT_DT,PREF_CONTACT_TIME,D_PINCODE,EDUCATIONAL_STATUS,MARITAL_STATUS,SPOUSE_NAME,SPOUSE_EMP_STATUS,DEPENDENT_CHILDREN,DEPENDENT_OTHERS,ACCOMODATION,EMPLOYMENT_STATUS,EMPLOYMENT_TENURE,RETIREMENT_AGE,PREV_DESIGNATION,PREV_EMPLOYER,DESIGNATION,SALARY,OTHER_INCOME,RENT,INSURANCE,LOAN_PAYMENT,OTHER_EXPENSES,HOUSE_VALUE,CREDIT_CARDS,E_COUNTRY,CCY_PERS_INCEXP
0,18030000,I,IKOYI,LAGOS,NG,NG,ENG,NG,3,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,3,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23401,N,MR,1966-08-15 00:00:00,N,M,R,N,E,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33020001,I,IKORODU,LAGOS,NG,NG,ENG,NG,6,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,6,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,MR,1950-01-01 00:00:00,N,M,R,N,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69090004,I,Lagos State,VICTORIA ISLAND,NG,NG,ENG,NG,14,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,14,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,N,Ms.,1970-04-10 00:00:00,N,F,R,N,E,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,81010006,I,SHAGAMU ROADIKORODU,LAGOS,NG,NG,ENG,NG,6,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,6,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,NaN,MR,1967-04-26 00:00:00,N,M,R,N,NaN,NaN,NG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71120007,I,SURULERE,LAGOS,NG,NG,ENG,NG,4,N,N,N,IND,NaN,0.0,0.0,0.0,NaN,NGN,NaN,FCUBSPAR,0.0,0.0,0.0,0.0,4,NaN,MAIL,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,234,N,MR,1965-09-24 00:00:00,N,M,R,N,M,NaN,NG,N,N,NaN,NaN,0.0,NaN,NaN,M,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
all_demographics.isnull().sum()/len(all_demographics)

CUSTOMER_UNIQUE_ID    0.000000
CUSTOMER_TYPE         0.000000
ADDRESS_LINE3         0.025543
ADDRESS_LINE4         0.151308
COUNTRY               0.000000
                        ...   
OTHER_EXPENSES        0.967501
HOUSE_VALUE           0.967501
CREDIT_CARDS          0.999722
E_COUNTRY             0.987423
CCY_PERS_INCEXP       0.999499
Length: 127, dtype: float64

In [0]:
#removing all columns with columns greater than 90% nan
#all_demographics.dropna(thresh=.9*len(all_demographics),axis=1,inplace=True)

In [0]:
#all_demographics=all_demographics_data.copy()

In [0]:
all_demographics=all_demographics.loc[:,all_demographics.isnull().mean() <= 0.2]

In [0]:
#removing some columns from all_demographics data
#removal was due to to many NaN values,and some the meaning was not given in data dictionary so i removed
all_demographics=all_demographics.drop(['ADDRESS_LINE4','EXT_REF_NO','CIF_CREATION_DATE','STAFF','PRIVATE_CUSTOMER','AR_AP_TRACKING','CUSTOMER_PREFIX','DATE_OF_BIRTH','AGE_PROOF_SUBMITTED','BIRTH_COUNTRY','LIMIT_CCY','FT_ACCTING_AS_OF','DEFAULT_MEDIA','MINOR'],axis=1)

In [207]:
all_dictionary[all_dictionary['FEATURES']=='MINOR']

,FEATURES,DESCRIPTION,DATA TYPE,Unnamed: 3,Unnamed: 4
116,MINOR,Minor,NaN,NaN,NaN


In [208]:
all_demographics.isnull().sum()

CUSTOMER_UNIQUE_ID            0
CUSTOMER_TYPE                 0
ADDRESS_LINE3               459
COUNTRY                       0
NATIONALITY                   0
LANGUAGE                      0
EXPOSURE_COUNTRY              0
LOCAL_BRANCH                  0
FROZEN                        0
DECEASED                      0
WHEREABOUTS_UNKNOWN           0
CUSTOMER_CATEGORY             0
LIAB_NODE                     0
LIAB_BR                       0
UTILITY_PROVIDER              0
AML_REQUIRED                  0
UNADVISED                     0
CONSOL_TAX_CERT_REQD          0
INDIVIDUAL_TAX_CERT_REQD      0
CLS_CCY_ALLOWED               0
CLS_PARTICIPANT               0
RP_CUSTOMER                   0
AUTO_CREATE_ACCOUNT           0
TRACK_LIMITS                  0
SEX                          20
MARITAL_STATUS              134
dtype: int64

In [209]:
all_demographics.isnull().sum().to_dict()

{'ADDRESS_LINE3': 459,
 'AML_REQUIRED': 0,
 'AUTO_CREATE_ACCOUNT': 0,
 'CLS_CCY_ALLOWED': 0,
 'CLS_PARTICIPANT': 0,
 'CONSOL_TAX_CERT_REQD': 0,
 'COUNTRY': 0,
 'CUSTOMER_CATEGORY': 0,
 'CUSTOMER_TYPE': 0,
 'CUSTOMER_UNIQUE_ID': 0,
 'DECEASED': 0,
 'EXPOSURE_COUNTRY': 0,
 'FROZEN': 0,
 'INDIVIDUAL_TAX_CERT_REQD': 0,
 'LANGUAGE': 0,
 'LIAB_BR': 0,
 'LIAB_NODE': 0,
 'LOCAL_BRANCH': 0,
 'MARITAL_STATUS': 134,
 'NATIONALITY': 0,
 'RP_CUSTOMER': 0,
 'SEX': 20,
 'TRACK_LIMITS': 0,
 'UNADVISED': 0,
 'UTILITY_PROVIDER': 0,
 'WHEREABOUTS_UNKNOWN': 0}

In [0]:
#meaning of minor
all_demographics.ADDRESS_LINE3.fillna("Unknown",inplace=True)

In [211]:
#meaning of ACY_AVL_BAL
all_dictionary[all_dictionary['FEATURES']=='TRACK_LIMITS']

,FEATURES,DESCRIPTION,DATA TYPE,Unnamed: 3,Unnamed: 4


In [0]:
all_demographics.SEX.fillna("Unknown",inplace=True)

In [0]:
all_demographics.MARITAL_STATUS.fillna("Unknown",inplace=True)

In [214]:
all_demographics.isnull().sum()

CUSTOMER_UNIQUE_ID          0
CUSTOMER_TYPE               0
ADDRESS_LINE3               0
COUNTRY                     0
NATIONALITY                 0
LANGUAGE                    0
EXPOSURE_COUNTRY            0
LOCAL_BRANCH                0
FROZEN                      0
DECEASED                    0
WHEREABOUTS_UNKNOWN         0
CUSTOMER_CATEGORY           0
LIAB_NODE                   0
LIAB_BR                     0
UTILITY_PROVIDER            0
AML_REQUIRED                0
UNADVISED                   0
CONSOL_TAX_CERT_REQD        0
INDIVIDUAL_TAX_CERT_REQD    0
CLS_CCY_ALLOWED             0
CLS_PARTICIPANT             0
RP_CUSTOMER                 0
AUTO_CREATE_ACCOUNT         0
TRACK_LIMITS                0
SEX                         0
MARITAL_STATUS              0
dtype: int64

In [0]:
#all_demographics_data,Need to set index to customer ID
#all_demographics_data treated
all_demographics_data=all_demographics.copy()

In [0]:
#convert index to int
all_demographics_data.index=all_demographics_data.index.astype('int')

In [0]:
all_demographics_data.sort_index(inplace=True)

In [218]:
'''
for i in columns:
    result=all_demographics[i].value_counts().to_list()
    percent=result[0]/sum(result)
    if percent>.9:
        all_demographics.drop(columns=i)'''

'\nfor i in columns:\n    result=all_demographics[i].value_counts().to_list()\n    percent=result[0]/sum(result)\n    if percent>.9:\n        all_demographics.drop(columns=i)'

In [0]:
all_loans_application.index=all_loans_application.index.astype('int')

In [0]:
all_loans_application.sort_index(inplace=True)

In [0]:
all_loans_application_data=all_loans_application.copy()

In [222]:
all_loans_application_data.drop(['OFFERID','DATE_OF_APPLY'],axis=1)

,INCR_ALLOWED,APPLIED_AMOUNT,APPLICATION_STATUS,MODULE,LOAN_CLASSIFICATION
CUSTOMER_UNIQUE_ID,,,,,
56000,46209.37,35000.0,D,AMPC,PERFORMING
62005,47280.70,50000.0,D,AMPC,PERFORMING
62005,47280.70,50000.0,D,AMPC,PERFORMING
104008,-5406.22,268000.0,D,STP,PERFORMING
175007,0.00,9000.0,D,LATO,PERFORMING
...,...,...,...,...,...
999823007,0.00,17000.0,D,LATO,PERFORMING
999823007,0.00,14000.0,D,LATO,NON-PERFORMING
999881002,36109.20,183635.0,D,STP,PERFORMING


In [223]:
##all_loans_application treated
all_loans_application['LOAN_CLASSIFICATION'].unique()

array(['PERFORMING', 'NON-PERFORMING'], dtype=object)

In [0]:
#all_loans_application_data.reset_index(inplace=True)

In [0]:
#target varriable
all_loans_application_data.loc[all_loans_application.LOAN_CLASSIFICATION=='PERFORMING','LOAN_CLASSIFICATION']=1

In [0]:
#target varriable
all_loans_application_data.loc[all_loans_application.LOAN_CLASSIFICATION=='NON-PERFORMING','LOAN_CLASSIFICATION']=0

In [227]:
#all_loans_application_data treated
all_loans_application_data.LOAN_CLASSIFICATION.unique()

array([1, 0])

In [0]:
all_investments.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [229]:
all_investments.loc[important_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,CONTRACT_REF_NO,PRODUCT,PRODUCT_NAME,PAYMENT_METHOD,CURRENCY,AMOUNT,LCY_AMOUNT,BOOKING_DATE,MATURITY_DATE,ROLLOVER_ALLOWED,TENOR,ROLLOVER_COUNT,CONTRACT_STATUS,MAIN_COMP_RATE,MASKED_ACCOUNT
CUSTOMER_UNIQUE_ID,,,,,,,,,,,,,,,
562966000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
867156003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
747196003,099TBCS180100002,TBCS,TBILLS INVT. FOR CUSTOMER,D,NGN,3000000.0,3000000.0,1/10/2018,6/21/2018,N,163.0,0.0,L,13.0,07******36
747196003,099TBCS180850020,TBCS,TBILLS INVT. FOR CUSTOMER,D,NGN,1000000.0,1000000.0,3/26/2018,8/2/2018,N,132.0,0.0,L,12.0,07******36
747196003,099TBCS181830515,TBCS,TBILLS INVT. FOR CUSTOMER,D,NGN,2712000.0,2712000.0,7/2/2018,11/15/2018,N,136.0,0.0,L,11.0,07******36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778317002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448867003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
760097005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#convert index to int
all_investments.index=all_investments.index.astype('int')

In [0]:
all_investments.sort_index(inplace=True)

In [232]:
all_investments.isna().sum()

CONTRACT_REF_NO     0
PRODUCT             0
PRODUCT_NAME        0
PAYMENT_METHOD      0
CURRENCY            0
AMOUNT              0
LCY_AMOUNT          0
BOOKING_DATE        0
MATURITY_DATE       1
ROLLOVER_ALLOWED    0
TENOR               1
ROLLOVER_COUNT      0
CONTRACT_STATUS     0
MAIN_COMP_RATE      0
MASKED_ACCOUNT      0
dtype: int64

In [0]:
#dropping all rows with null values
all_investments=all_investments.dropna(how='any',axis=0)

In [234]:
#all_investment_data treated
all_investments.isnull().sum()

CONTRACT_REF_NO     0
PRODUCT             0
PRODUCT_NAME        0
PAYMENT_METHOD      0
CURRENCY            0
AMOUNT              0
LCY_AMOUNT          0
BOOKING_DATE        0
MATURITY_DATE       0
ROLLOVER_ALLOWED    0
TENOR               0
ROLLOVER_COUNT      0
CONTRACT_STATUS     0
MAIN_COMP_RATE      0
MASKED_ACCOUNT      0
dtype: int64

In [0]:
#all_investments_data

In [0]:
all_investments['BOOKING_DATE']=pd.to_datetime(all_investments['BOOKING_DATE'])
all_investments['MATURITY_DATE']=pd.to_datetime(all_investments['MATURITY_DATE'])

In [0]:
#function to find difference column for new date
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
def Time_Diff(time1,time2):
  return np.timedelta64((time2-time1),'D')/np.timedelta64(1,'D')

In [0]:
all_investments['date_diff']=all_investments.apply(lambda x :Time_Diff(x['BOOKING_DATE'],x['MATURITY_DATE']),axis=1)

In [0]:
all_investments=all_investments.drop(['MATURITY_DATE','BOOKING_DATE'],axis=1)

In [240]:
all_investments.head()

,CONTRACT_REF_NO,PRODUCT,PRODUCT_NAME,PAYMENT_METHOD,CURRENCY,AMOUNT,LCY_AMOUNT,ROLLOVER_ALLOWED,TENOR,ROLLOVER_COUNT,CONTRACT_STATUS,MAIN_COMP_RATE,MASKED_ACCOUNT,date_diff
CUSTOMER_UNIQUE_ID,,,,,,,,,,,,,,
226008,099MTDB193305017,MTDB,TERM DEPOSITS,B,NGN,200000.0,200000.0,Y,60.0,0,A,5.5,00******04,60.0
226008,099MTDB193170028,MTDB,TERM DEPOSITS,B,NGN,1000000.0,1000000.0,Y,30.0,0,A,5.5,00******49,30.0
1321003,099TBCS170190008,TBCS,TBILLS INVT. FOR CUSTOMER,D,NGN,1100000.0,1100000.0,N,364.0,0,L,18.5,00******36,364.0
1352000,099COMP171000003,COMP,COMMERCIAL PAPERS SOLD,D,NGN,450000.0,450000.0,Y,31.0,0,L,6.5,00******51,28.0
2366007,099MTDB19291A00G,MTDB,TERM DEPOSITS,B,NGN,200000.0,200000.0,Y,30.0,0,L,5.0,00******55,30.0


In [241]:
all_investments.loc[747196003]

,CONTRACT_REF_NO,PRODUCT,PRODUCT_NAME,PAYMENT_METHOD,CURRENCY,AMOUNT,LCY_AMOUNT,ROLLOVER_ALLOWED,TENOR,ROLLOVER_COUNT,CONTRACT_STATUS,MAIN_COMP_RATE,MASKED_ACCOUNT,date_diff
CUSTOMER_UNIQUE_ID,,,,,,,,,,,,,,
747196003,099TBCS180100002,TBCS,TBILLS INVT. FOR CUSTOMER,D,NGN,3000000.0,3000000.0,N,163.0,0,L,13.00,07******36,162.0
747196003,099TBCS183330563,TBCS,TBILLS INVT. FOR CUSTOMER,D,NGN,5500000.0,5500000.0,N,175.0,0,L,12.65,07******36,175.0
747196003,099TBCS182680589,TBCS,TBILLS INVT. FOR CUSTOMER,D,NGN,2000000.0,2000000.0,N,65.0,0,L,10.50,07******36,65.0
747196003,099TBCS180850020,TBCS,TBILLS INVT. FOR CUSTOMER,D,NGN,1000000.0,1000000.0,N,132.0,0,L,12.00,07******36,129.0
747196003,099TBCS181830515,TBCS,TBILLS INVT. FOR CUSTOMER,D,NGN,2712000.0,2712000.0,N,136.0,0,L,11.00,07******36,136.0


In [0]:
#Seeing from the above info that documents all investments per customer.taking the 
#mean investments of the customer will be a better options
#convert each of maturity date and  booking date in days.find the difference.and while grouping by find the mean days
all_investments=all_investments.groupby(all_investments.index).agg({'AMOUNT':'mean','TENOR':'mean','MAIN_COMP_RATE':'mean','date_diff':'mean'})


In [243]:
all_investments.head()

,AMOUNT,TENOR,MAIN_COMP_RATE,date_diff
CUSTOMER_UNIQUE_ID,,,,
226008,600000.0,45.0,5.5,45.0
1321003,1100000.0,364.0,18.5,364.0
1352000,450000.0,31.0,6.5,28.0
2366007,150000.0,30.0,5.0,30.0
2471001,1000000.0,247.0,15.0,247.0


In [0]:
#all_investments_data Treated
all_investments_data=all_investments.copy()

In [245]:
#all_investments_data treated
all_investments_data.isnull().sum()

AMOUNT            0
TENOR             0
MAIN_COMP_RATE    0
date_diff         0
dtype: int64

In [0]:
all_loans.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [0]:
#convert index to int
all_loans.index=all_loans.index.astype('int')

In [0]:
all_loans.sort_index(inplace=True)

In [249]:
all_loans.head()

,LOAN_REF,PRODUCT_CODE,PRODUCT_NAME,BOOK_DATE,MATURITY_DATE,AMOUNT_FINANCED,CURRENCY,ACCOUNT_STATUS,MASKED_ACCOUNT
CUSTOMER_UNIQUE_ID,,,,,,,,,
56000,099AMPC190321415,AMPC,ACCESS MONEY PAYDAY ADVANCE,2/1/2019,3/3/2019,24000.0,NGN,L,00******73
56000,099PDLP1930305Y2,PDLP,PAYDAY ADVANCE,10/30/2019,11/29/2019,135000.0,NGN,A,00******73
56000,099PDLP191513543,PDLP,PAYDAY ADVANCE,5/31/2019,6/30/2019,126000.0,NGN,L,00******73
56000,099PDLP190880084,PDLP,PAYDAY ADVANCE,3/29/2019,4/28/2019,10000.0,NGN,L,00******73
56000,099AMPC183333164,AMPC,ACCESS MONEY PAYDAY ADVANCE,11/29/2018,12/29/2018,18000.0,NGN,L,00******73


In [250]:
all_loans.loc[important_indexes]

,LOAN_REF,PRODUCT_CODE,PRODUCT_NAME,BOOK_DATE,MATURITY_DATE,AMOUNT_FINANCED,CURRENCY,ACCOUNT_STATUS,MASKED_ACCOUNT
CUSTOMER_UNIQUE_ID,,,,,,,,,
562966000,099AMPC173270599,AMPC,ACCESS MONEY PAYDAY ADVANCE,11/23/2017,12/23/2017,150000.0,NGN,L,00******32
562966000,099AMPC183093515,AMPC,ACCESS MONEY PAYDAY ADVANCE,11/5/2018,12/5/2018,300000.0,NGN,L,00******32
562966000,099PDLP190600827,PDLP,PAYDAY ADVANCE,3/1/2019,3/31/2019,500000.0,NGN,L,00******32
562966000,099AMPC180740112,AMPC,ACCESS MONEY PAYDAY ADVANCE,3/15/2018,4/14/2018,30000.0,NGN,L,00******32
562966000,099AMPC171671301,AMPC,ACCESS MONEY PAYDAY ADVANCE,6/16/2017,7/16/2017,200000.0,NGN,L,00******32
...,...,...,...,...,...,...,...,...,...
65167007,099AMPC173521158,AMPC,ACCESS MONEY PAYDAY ADVANCE,12/18/2017,1/17/2018,35000.0,NGN,L,00******06
65167007,099SADV191780030,SADV,ONLINE SALARY ADVANCE,6/27/2019,9/25/2019,98000.0,NGN,L,00******06
65167007,099PDLP192265220,PDLP,PAYDAY ADVANCE,8/14/2019,9/13/2019,20000.0,NGN,L,00******06


In [251]:
all_loans.isnull().sum()/len(all_loans)

LOAN_REF           0.0
PRODUCT_CODE       0.0
PRODUCT_NAME       0.0
BOOK_DATE          0.0
MATURITY_DATE      0.0
AMOUNT_FINANCED    0.0
CURRENCY           0.0
ACCOUNT_STATUS     0.0
MASKED_ACCOUNT     0.0
dtype: float64

In [0]:
#convert datecolumns to date type
all_loans.MATURITY_DATE=pd.to_datetime(all_loans.MATURITY_DATE)
all_loans.BOOK_DATE=pd.to_datetime(all_loans.BOOK_DATE)


In [0]:
all_loans['date_diff1']=(all_loans.MATURITY_DATE-all_loans.BOOK_DATE)/np.timedelta64(1,'D')

In [0]:
all_loans.index=all_loans.index.astype('int')


In [0]:
all_loans.sort_index(inplace=True)

In [0]:
#aggregating the account status using mode 
account_status=all_loans.groupby(all_loans.index).ACCOUNT_STATUS.apply(pd.Series.mode).sort_index().to_frame()

In [0]:
amount=all_loans.groupby(all_loans.index).AMOUNT_FINANCED.mean().sort_index().to_frame()

In [0]:
product_code=all_loans.groupby(all_loans.index).PRODUCT_CODE.apply(pd.Series.mode).sort_index().to_frame()

In [0]:
date_diff1=all_loans.groupby(all_loans.index).date_diff1.mean().sort_index().to_frame()

In [0]:
#important columns from all_loans
dfs=[amount,date_diff1]

In [0]:
all_loans_data=pd.concat(dfs,axis=1)

In [262]:
#dropping date_diff1 since the column already exit in all_loans_application
all_loans_data.drop('date_diff1',axis=1)

,AMOUNT_FINANCED
CUSTOMER_UNIQUE_ID,
56000,56827.586207
62005,72074.074074
104008,46111.111111
175007,5235.294118
200006,74937.500000
...,...
999737005,2323.529412
999823007,8500.000000
999881002,137183.874000


In [0]:
#setting index of all_transactions
all_transactions_digital.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [0]:
#convert index to int
all_transactions_digital.index=all_transactions_digital.index.astype('int')

In [0]:
all_transactions_digital.sort_index(inplace=True)

In [266]:
all_transactions_digital.head()

,MASKED_ACCT,TRN_REF_NO,AC_CCY,DRCR_IND,TRN_CODE,FCY_AMOUNT,EXCH_RATE,LCY_AMOUNT,TRN_DT
CUSTOMER_UNIQUE_ID,,,,,,,,,
56000,00******73,098AWRV192730K2Z,NGN,D,FFR,NaN,NaN,65.0,2019-09-30 00:00:00
56000,00******73,099MJNL19028I1X3,NGN,D,PWC,NaN,NaN,20.0,2019-01-29 00:00:00
56000,00******73,040ACWO19119001i,NGN,D,CWU,NaN,NaN,40000.0,2019-04-29 00:00:00
56000,00******73,099MJNL1902959RG,NGN,D,PWC,NaN,NaN,20.0,2019-01-29 00:00:00
56000,00******73,322ACWO19029003L,NGN,D,CWU,NaN,NaN,14000.0,2019-01-29 00:00:00


In [267]:
all_transactions_digital=all_transactions_digital.loc[important_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [268]:
all_transactions_digital.head()

,MASKED_ACCT,TRN_REF_NO,AC_CCY,DRCR_IND,TRN_CODE,FCY_AMOUNT,EXCH_RATE,LCY_AMOUNT,TRN_DT
CUSTOMER_UNIQUE_ID,,,,,,,,,
562966000,06******15,09901kt192482422,NGN,D,SRC,NaN,NaN,11527.84,2019-09-05 00:00:00
562966000,00******32,098WNVV191610BH8,NGN,D,FWR,NaN,NaN,1600.00,2019-06-10 00:00:00
562966000,00******32,099ZTRF190024USN,NGN,D,DBC,0.0,NaN,1000.00,2019-01-02 00:00:00
562966000,00******32,099MJRL192763hvY,NGN,D,QTM,NaN,NaN,2220.00,2019-10-03 00:00:00
562966000,00******32,029INMC192760033,NGN,C,IAT,NaN,NaN,39675.00,2019-10-03 00:00:00


In [269]:
all_transactions_digital.isnull().sum()/len(all_transactions_digital)

MASKED_ACCT    0.000018
TRN_REF_NO     0.000018
AC_CCY         0.000018
DRCR_IND       0.000018
TRN_CODE       0.000018
FCY_AMOUNT     0.868479
EXCH_RATE      0.999154
LCY_AMOUNT     0.000018
TRN_DT         0.000018
dtype: float64

In [0]:
#removing all null columns greater than 80%.living ones less than 80%
all_transactions_digital=all_transactions_digital.loc[:,all_transactions_digital.isnull().mean() < 0.8]

In [271]:
all_transactions_digital.isnull().sum()/len(all_transactions_digital)

MASKED_ACCT    0.000018
TRN_REF_NO     0.000018
AC_CCY         0.000018
DRCR_IND       0.000018
TRN_CODE       0.000018
LCY_AMOUNT     0.000018
TRN_DT         0.000018
dtype: float64

In [272]:
all_transactions_digital.dropna(how='any',axis=0)

,MASKED_ACCT,TRN_REF_NO,AC_CCY,DRCR_IND,TRN_CODE,LCY_AMOUNT,TRN_DT
CUSTOMER_UNIQUE_ID,,,,,,,
562966000,06******15,09901kt192482422,NGN,D,SRC,11527.84,2019-09-05 00:00:00
562966000,00******32,098WNVV191610BH8,NGN,D,FWR,1600.00,2019-06-10 00:00:00
562966000,00******32,099ZTRF190024USN,NGN,D,DBC,1000.00,2019-01-02 00:00:00
562966000,00******32,099MJRL192763hvY,NGN,D,QTM,2220.00,2019-10-03 00:00:00
562966000,00******32,029INMC192760033,NGN,C,IAT,39675.00,2019-10-03 00:00:00
...,...,...,...,...,...,...,...
65167007,00******06,098WNVI190880BI9,NGN,D,FWR,3000.00,2019-03-29 00:00:00
65167007,00******06,098WNVI1918607dV,NGN,D,FWR,5000.00,2019-07-05 00:00:00
65167007,00******06,098WNVI1909109UO,NGN,D,FWR,4000.00,2019-04-01 00:00:00


In [273]:
all_transactions_digital.isnull().sum()

MASKED_ACCT    163
TRN_REF_NO     163
AC_CCY         163
DRCR_IND       163
TRN_CODE       163
LCY_AMOUNT     163
TRN_DT         163
dtype: int64

In [0]:
#out of the 7 fields i decided to pick masked account to check the row index that are null
all_transactions_digital_idx=all_transactions_digital.loc[all_transactions_digital['MASKED_ACCT'].isnull()].index

In [275]:
#indexes where there were null in the rows
all_transactions_digital_idx

Int64Index([688775005, 916490004, 829948005, 892728000, 627506006, 845721009,
            905122006, 207407006, 816232009, 711015007,
            ...
            336561003, 611594006, 314075001,  53353011, 365834012, 105029001,
             22594016, 832199003, 190903002, 162998000],
           dtype='int64', name='CUSTOMER_UNIQUE_ID', length=163)

In [0]:
#drop row index with null values
all_transactions_digital=all_transactions_digital.drop(all_transactions_digital_idx.tolist())

In [0]:
#all_transactions_digital_data treated
all_transactions_digital_data=all_transactions_digital.copy()

In [278]:
all_transactions_digital_data.head()

,MASKED_ACCT,TRN_REF_NO,AC_CCY,DRCR_IND,TRN_CODE,LCY_AMOUNT,TRN_DT
CUSTOMER_UNIQUE_ID,,,,,,,
562966000,06******15,09901kt192482422,NGN,D,SRC,11527.84,2019-09-05 00:00:00
562966000,00******32,098WNVV191610BH8,NGN,D,FWR,1600.00,2019-06-10 00:00:00
562966000,00******32,099ZTRF190024USN,NGN,D,DBC,1000.00,2019-01-02 00:00:00
562966000,00******32,099MJRL192763hvY,NGN,D,QTM,2220.00,2019-10-03 00:00:00
562966000,00******32,029INMC192760033,NGN,C,IAT,39675.00,2019-10-03 00:00:00


In [279]:
all_transactions_digital.loc[562966000]

,MASKED_ACCT,TRN_REF_NO,AC_CCY,DRCR_IND,TRN_CODE,LCY_AMOUNT,TRN_DT
CUSTOMER_UNIQUE_ID,,,,,,,
562966000,06******15,09901kt192482422,NGN,D,SRC,11527.84,2019-09-05 00:00:00
562966000,00******32,098WNVV191610BH8,NGN,D,FWR,1600.00,2019-06-10 00:00:00
562966000,00******32,099ZTRF190024USN,NGN,D,DBC,1000.00,2019-01-02 00:00:00
562966000,00******32,099MJRL192763hvY,NGN,D,QTM,2220.00,2019-10-03 00:00:00
562966000,00******32,029INMC192760033,NGN,C,IAT,39675.00,2019-10-03 00:00:00
...,...,...,...,...,...,...,...
562966000,00******32,098WNVV191640ZEk,NGN,D,FWR,7000.00,2019-06-14 00:00:00
562966000,00******32,099ZTRF192631IXK,NGN,C,LDC,10000.00,2019-09-20 00:00:00
562966000,00******32,098WNVV191220LzW,NGN,D,FWR,6900.00,2019-05-02 00:00:00


In [280]:
#trying to check meaning of DRCR_IND
all_dictionary[all_dictionary['FEATURES']=='DRCR_IND']

,FEATURES,DESCRIPTION,DATA TYPE,Unnamed: 3,Unnamed: 4
26,DRCR_IND,"Debit/Credit indicator, D- Debit, C - Credit",CHAR,NaN,NaN
66,DRCR_IND,"Debit/Credit indicator, D- Debit, C - Credit",CHAR,NaN,NaN


In [0]:
all_transactions_digital=all_transactions_digital.groupby([all_transactions_digital.index,all_transactions_digital.DRCR_IND])[['LCY_AMOUNT']].sum()

In [0]:
all_transactions_digital=pd.pivot_table(all_transactions_digital.reset_index(),index='CUSTOMER_UNIQUE_ID',columns='DRCR_IND', values='LCY_AMOUNT',aggfunc=np.sum).reset_index()[['CUSTOMER_UNIQUE_ID', 'C', 'D']]

In [283]:
all_transactions_digital.isnull().sum()

DRCR_IND
CUSTOMER_UNIQUE_ID     0
C                     71
D                      7
dtype: int64

In [0]:
# pick C to check the null index
all_transactions_digital_null_idxC=all_transactions_digital.loc[all_transactions_digital['C'].isnull()].index

In [0]:
#drop row index with null values in C column
all_transactions_digital=all_transactions_digital.drop(all_transactions_digital_null_idxC.tolist())

In [0]:
#out of the 7 fields i decided to pick masked account to check the null index
all_transactions_digital_null_idxD=all_transactions_digital.loc[all_transactions_digital['D'].isnull()].index

In [0]:
#drop row index with null values in D column
all_transactions_digital=all_transactions_digital.drop(all_transactions_digital_null_idxD.tolist())

In [288]:
all_transactions_digital.isnull().sum()

DRCR_IND
CUSTOMER_UNIQUE_ID    0
C                     0
D                     0
dtype: int64

In [0]:
#treated
all_transactions_digital_data=all_transactions_digital.copy()

In [0]:
#bureau_score.set_index('CUSTOMER_UNIQUE_ID',inplace=True)


In [0]:
#bureau_score.sort_index(inplace=True)

In [0]:
#bureau_score=bureau_score.loc[important_indexes]

In [0]:
#bureau_score_data=bureau_score.copy()

In [0]:
dff=[all_loans_application_data,all_loans_data,all_investments_data,all_balances_data,all_accounts_data,all_demographics_data]

In [0]:
#train=dff[0].join(dff[1:])

In [0]:
#train.isnull().sum()/len(train)

In [0]:
#train.index

In [0]:
from functools import reduce
train=reduce(lambda x,y: pd.merge(x,y, on='CUSTOMER_UNIQUE_ID', how='outer'), dff)

In [298]:
train.isnull().sum()/len(train)

CUSTOMER_UNIQUE_ID          0.000000
OFFERID                     0.000000
INCR_ALLOWED                0.000000
APPLIED_AMOUNT              0.000000
DATE_OF_APPLY               0.000000
APPLICATION_STATUS          0.000000
MODULE                      0.000000
LOAN_CLASSIFICATION         0.000000
AMOUNT_FINANCED             0.000000
date_diff1                  0.000000
AMOUNT                      0.950601
TENOR                       0.950601
MAIN_COMP_RATE              0.950601
date_diff                   0.950601
MASKED_ACCOUNT              0.288509
VAL_DT                      0.288509
BAL                         0.288509
ACY_AVL_BAL                 0.000000
CUSTOMER_TYPE               0.000817
ADDRESS_LINE3               0.000817
COUNTRY                     0.000817
NATIONALITY                 0.000817
LANGUAGE                    0.000817
EXPOSURE_COUNTRY            0.000817
LOCAL_BRANCH                0.000817
FROZEN                      0.000817
DECEASED                    0.000817
W

In [0]:
#removing all null columns greater than 80%.living ones less than 80%
train=train.loc[:,train.isnull().mean() < 0.05]

In [300]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34252 entries, 0 to 34251
Data columns (total 36 columns):
CUSTOMER_UNIQUE_ID          34252 non-null int64
OFFERID                     34252 non-null object
INCR_ALLOWED                34252 non-null float64
APPLIED_AMOUNT              34252 non-null float64
DATE_OF_APPLY               34252 non-null object
APPLICATION_STATUS          34252 non-null object
MODULE                      34252 non-null object
LOAN_CLASSIFICATION         34252 non-null int64
AMOUNT_FINANCED             34252 non-null float64
date_diff1                  34252 non-null float64
ACY_AVL_BAL                 34252 non-null float64
CUSTOMER_TYPE               34224 non-null object
ADDRESS_LINE3               34224 non-null object
COUNTRY                     34224 non-null object
NATIONALITY                 34224 non-null object
LANGUAGE                    34224 non-null object
EXPOSURE_COUNTRY            34224 non-null object
LOCAL_BRANCH                34224 non-

In [301]:
train.isnull().sum()/len(train)

CUSTOMER_UNIQUE_ID          0.000000
OFFERID                     0.000000
INCR_ALLOWED                0.000000
APPLIED_AMOUNT              0.000000
DATE_OF_APPLY               0.000000
APPLICATION_STATUS          0.000000
MODULE                      0.000000
LOAN_CLASSIFICATION         0.000000
AMOUNT_FINANCED             0.000000
date_diff1                  0.000000
ACY_AVL_BAL                 0.000000
CUSTOMER_TYPE               0.000817
ADDRESS_LINE3               0.000817
COUNTRY                     0.000817
NATIONALITY                 0.000817
LANGUAGE                    0.000817
EXPOSURE_COUNTRY            0.000817
LOCAL_BRANCH                0.000817
FROZEN                      0.000817
DECEASED                    0.000817
WHEREABOUTS_UNKNOWN         0.000817
CUSTOMER_CATEGORY           0.000817
LIAB_NODE                   0.000817
LIAB_BR                     0.000817
UTILITY_PROVIDER            0.000817
AML_REQUIRED                0.000817
UNADVISED                   0.000817
C

In [0]:

train.LOCAL_BRANCH.fillna("Unknown",inplace=True)
train.SEX.fillna("prefer not to say",inplace=True)
train.CUSTOMER_CATEGORY.fillna("unknown",inplace=True)
train.MARITAL_STATUS.fillna('unknown',inplace=True)
train.ADDRESS_LINE3.fillna('unknown',inplace=True)



In [0]:
train.drop(columns=['AUTO_CREATE_ACCOUNT'
                    ,'FROZEN'                      
                    ,'DECEASED'                     
                    ,'WHEREABOUTS_UNKNOWN'
                    ,'UTILITY_PROVIDER'
                    ,'AML_REQUIRED'
                    ,'INDIVIDUAL_TAX_CERT_REQD'
                    ,'UNADVISED'
                    ,'CONSOL_TAX_CERT_REQD'
                    ,'CLS_CCY_ALLOWED'
                    ,'CLS_PARTICIPANT'
                    ,'TRACK_LIMITS',
                    'RP_CUSTOMER',
                    'EXPOSURE_COUNTRY',
                    'CUSTOMER_TYPE',
                    'LIAB_BR'
                    ,'LIAB_NODE']
           ,inplace=True)

In [304]:
train.isnull().sum()/len(train)

CUSTOMER_UNIQUE_ID     0.000000
OFFERID                0.000000
INCR_ALLOWED           0.000000
APPLIED_AMOUNT         0.000000
DATE_OF_APPLY          0.000000
APPLICATION_STATUS     0.000000
MODULE                 0.000000
LOAN_CLASSIFICATION    0.000000
AMOUNT_FINANCED        0.000000
date_diff1             0.000000
ACY_AVL_BAL            0.000000
ADDRESS_LINE3          0.000000
COUNTRY                0.000817
NATIONALITY            0.000817
LANGUAGE               0.000817
LOCAL_BRANCH           0.000000
CUSTOMER_CATEGORY      0.000000
SEX                    0.000000
MARITAL_STATUS         0.000000
dtype: float64

In [0]:
train.drop(columns=['NATIONALITY','LANGUAGE','COUNTRY'],inplace=True)

In [306]:
train.isnull().sum()

CUSTOMER_UNIQUE_ID     0
OFFERID                0
INCR_ALLOWED           0
APPLIED_AMOUNT         0
DATE_OF_APPLY          0
APPLICATION_STATUS     0
MODULE                 0
LOAN_CLASSIFICATION    0
AMOUNT_FINANCED        0
date_diff1             0
ACY_AVL_BAL            0
ADDRESS_LINE3          0
LOCAL_BRANCH           0
CUSTOMER_CATEGORY      0
SEX                    0
MARITAL_STATUS         0
dtype: int64

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [0]:
le={}
for i in train.select_dtypes(include='object').columns:
    le[i]=LabelEncoder()
    train[i]=le[i].fit_transform(train[i].astype('str'))

In [0]:
y=train.pop('LOAN_CLASSIFICATION')

In [0]:
X_train,X_test,y_train,y_test=train_test_split(train,y,test_size=.3,random_state=234,stratify=y)



In [0]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from xgboost import XGBClassifier 
import xgboost as xgb
from sklearn.metrics import *

In [0]:
'''def get_model(X_train, X_test, y_train, y_test, model):

    scaler = MinMaxScaler()
    x_train_pre = scaler.fit_transform(X_train)
    x_test_pre = scaler.transform(X_test)
              
    # y_preprocessing = StandardScaler()
    # y_preprocessing.fit(y_train)
    y_train_pre = y_train
    y_test_pre = y_test

        
    if model == 'RFC':
        model = RandomForestClassifier(n_estimators=200)
        model.fit(x_train_pre,y_train_pre)
        y_prob = model.predict(x_test_pre)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_pre, y_prob)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        print("ROC:", roc_auc)
        
    if model == 'XGB':
        model = xgb.XGBClassifier(n_estimators=300)
        model.fit(x_train_pre,y_train_pre) 
        y_prob = model.predict(x_test_pre)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_pre, y_prob)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        print("ROC:", roc_auc)'''

  

In [325]:
scaler = MinMaxScaler()
x_train_pre = scaler.fit_transform(X_train)
x_test_pre = scaler.transform(X_test)
              
    # y_preprocessing = StandardScaler()
    # y_preprocessing.fit(y_train)
y_train_pre = y_train
y_test_pre = y_test

modelRF = RandomForestClassifier(n_estimators=200)
modelRF.fit(x_train_pre,y_train_pre)
y_probRF = modelRF.predict(x_test_pre)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_pre, y_probRF)
roc_aucRF = auc(false_positive_rate, true_positive_rate)
print("ROC:", roc_aucRF)

     

ROC: 0.8166494427256888


In [326]:
modelxgb =xgb.XGBClassifier(n_estimators=300)
modelxgb.fit(x_train_pre,y_train_pre) 
y_probXGB = modelxgb.predict(x_test_pre)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_pre, y_probXGB)
roc_aucXGB = auc(false_positive_rate, true_positive_rate)
print("ROC:", roc_aucXGB)

ROC: 0.5222265702780909


In [327]:
f1_score(y_pred,y_test)

NameError: ignored